## WeatherPy

#### In this example, you'll be creating a Python script to visualize the weather of 500+ cities across the world of varying distance from the equator. To accomplish this, you'll be utilizing a [simple Python library](https://pypi.python.org/pypi/citipy), the [OpenWeatherMap API](https://openweathermap.org/api), and a little common sense to create a representative model of weather across world cities.

#### Your objective is to build a series of scatter plots to showcase the following relationships:

* Temperature (F) vs. Latitude
* Humidity (%) vs. Latitude
* Cloudiness (%) vs. Latitude
* Wind Speed (mph) vs. Latitude

#### Your final notebook must:

* Randomly select **at least** 500 unique (non-repeat) cities based on latitude and longitude.
* Perform a weather check on each of the cities using a series of successive API calls.
* Include a print log of each city as it's being processed with the city number and city name.
* Save both a CSV of all data retrieved and png images for each scatter plot.

#### As final considerations:

* You must complete your analysis using a Jupyter notebook.
* You must use the Matplotlib or Pandas plotting libraries.
* You must include a written description of three observable trends based on the data.
* You must use proper labeling of your plots, including aspects like: Plot Titles (with date of analysis) and Axes Labels.
* See [Example Solution](WeatherPy_Example.pdf) for a reference on expected format.

## Hints and Considerations

* The city data is generated based on random coordinates; as such, your outputs will not be an exact match to the provided starter notebook.

* You may want to start this assignment by refreshing yourself on the [geographic coordinate system](http://desktop.arcgis.com/en/arcmap/10.3/guide-books/map-projections/about-geographic-coordinate-systems.htm).

* Next, spend the requisite time necessary to study the OpenWeatherMap API. Based on your initial study, you should be able to answer  basic questions about the API: Where do you request the API key? Which Weather API in particular will you need? What URL endpoints does it expect? What JSON structure does it respond with? Before you write a line of code, you should be aiming to have a crystal clear understanding of your intended outcome.

* A starter code for Citipy has been provided. However, if you're craving an extra challenge, push yourself to learn how it works: [citipy Python library](https://pypi.python.org/pypi/citipy). Before you try to incorporate the library into your analysis, start by creating simple test cases outside your main script to confirm that you are using it correctly. Too often, when introduced to a new library, students get bogged down by the most minor of errors -- spending hours investigating their entire code -- when, in fact, a simple and focused test would have shown their basic utilization of the library was wrong from the start. Don't let this be you!

* Part of our expectation in this challenge is that you will use critical thinking skills to understand how and why we're recommending the tools we are. What is Citipy for? Why would you use it in conjunction with the OpenWeatherMap API? How would you do so?

* In building your script, pay attention to the cities you are using in your query pool. Are you getting coverage of the full gamut of latitudes and longitudes? Or are you simply choosing 500 cities concentrated in one region of the world? Even if you were a geographic genius, simply rattling 500 cities based on your human selection would create a biased dataset. Be thinking of how you should counter this. (Hint: Consider the full range of latitudes).

* Lastly, remember -- this is a challenging activity. Push yourself! If you complete this task, then you can safely say that you've gained a strong mastery of the core foundations of data analytics and it will only go better from here. Good luck!

# WeatherPy
----

### Analysis
* As expected, the weather becomes significantly warmer as one approaches the equator (0 Deg. Latitude). More interestingly, however, is the fact that the southern hemisphere tends to be warmer this time of year than the northern hemisphere. This may be due to the tilt of the earth.
* There is no strong relationship between latitude and cloudiness. However, it is interesting to see that a strong band of cities sits at 0, 80, and 100% cloudiness.
* There is no strong relationship between latitude and wind speed. However, in northern hemispheres there is a flurry of cities with over 20 mph of wind.

---

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Actions in terminal before we begin

# pip install citipy
# add Weather Map api_key to config

# Dependencies and Setup
import openweathermapy.core as owm
import matplotlib.pyplot as plt
import json
import pandas as pd
import numpy as np
import requests
import time

# Import API key
from api_keys import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        city = city.title()
        cities.append(city)

# Print the city count to confirm sufficient count
# Do we need to make city names Capitalized?
print(cities)



['Leningradskiy', 'Khatanga', 'Pangnirtung', 'Punta Arenas', 'Mar Del Plata', 'Mar Del Plata', 'Mehamn', 'Sentyabrskiy', 'Iqaluit', 'Port Lincoln', 'Kapaa', 'Tasiilaq', 'Samarai', 'Dikson', 'Ambulu', 'Fortuna', 'Port Augusta', 'Klaksvik', 'Narsaq', 'Narsaq', 'Auki', 'Kapaa', 'Saint-Francois', 'Ponta Do Sol', 'Jamestown', 'Yellowknife', 'Ostrovnoy', 'Keshan', 'Hunza', 'Qaanaaq', 'Chapais', 'Bredasdorp', 'Ampanihy', 'Yellowknife', 'Nieves', 'Oranjemund', 'Isiro', 'Saskylakh', 'New Norfolk', 'Tasiilaq', 'Rikitea', 'Bluff', 'Port Alfred', 'Barrow', 'Albany', 'New Norfolk', 'Tasbuget', 'Palana', 'Victoria', 'Georgetown', 'Bundu', 'Castro', 'Lamu', 'Katsuura', 'Olafsvik', 'Kaitangata', 'Mataura', 'Biak', 'Souillac', 'Kapaa', 'San Rafael Del Sur', 'Rodrigues Alves', 'Ngunguru', 'Hilo', 'Bengkulu', 'Goldsboro', 'Kavaratti', 'Pevek', 'Dikson', 'Studenicani', 'Busselton', 'Asau', 'Tirumullaivasal', 'Barrow', 'Faya', 'Namibe', 'Hermanus', 'East London', 'Atuona', 'Hobart', 'Mar Del Plata', 'Perth

In [3]:
# Save config information.
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

# Build partial query URL
query_url = f"{url}appid={api_key}&units={units}&q="
query_url


'http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q='

In [17]:
city = []
lat = []
lon = []
temp_f = []
humidity = []
cloudiness = []
wind_mph = []

print(f'Begin Data Retrival: Print Processing Record, City, & API URL')
print('______________________________________________________________')
print('')

record = 0
setnum = 1

for city in cities:
    
    if (record > 50):
        record = 0
        setnum = setnum + 1
        
    print(f'Processing Record {record}; Set #{setnum} | {city}')
    record = record + 1
    #city = city.replace(' .', '%20', n=-1, case=None, flags=0, regex=True)
    city_url = query_url + city.replace(" ", "%20")
    print(city_url)
    
    response = requests.get(query_url + city).json()
    
    if response['cod'] == 200: #checks if response is sucessfull, if other than 200, skip code below
        lat.append(response['coord']['lat'])
        lon.append(response['coord']['lon'])
        temp_f.append(response['main']['temp'])
        humidity.append(response['main']['humidity'])
        cloudiness.append(response['clouds']['all'])
        wind_mph.append(response['wind']['speed'])
        
    total_recs = record * setnum


print(f'The total number of records called upon is {total_recs}')

Begin Data Retrival: Print Processing Record, City, & API URL
______________________________________________________________

Processing Record 0; Set #1 | Leningradskiy
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Leningradskiy
Processing Record 1; Set #1 | Khatanga
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Khatanga
Processing Record 2; Set #1 | Pangnirtung
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Pangnirtung
Processing Record 3; Set #1 | Punta Arenas
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Punta%20Arenas
Processing Record 4; Set #1 | Mar Del Plata
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Mar%20Del%20Plata
Processing Record 5; Set #1 | Mar Del Plata
http://api.openweathermap.org/data/

Processing Record 3; Set #2 | Olafsvik
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Olafsvik
Processing Record 4; Set #2 | Kaitangata
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Kaitangata
Processing Record 5; Set #2 | Mataura
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Mataura
Processing Record 6; Set #2 | Biak
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Biak
Processing Record 7; Set #2 | Souillac
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Souillac
Processing Record 8; Set #2 | Kapaa
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Kapaa
Processing Record 9; Set #2 | San Rafael Del Sur
http://api.openweathermap.org/data/2.5/weather?appid=cf61

Processing Record 8; Set #3 | Port Elizabeth
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Port%20Elizabeth
Processing Record 9; Set #3 | Bengkulu
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Bengkulu
Processing Record 10; Set #3 | Qaanaaq
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Qaanaaq
Processing Record 11; Set #3 | Mogadishu
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Mogadishu
Processing Record 12; Set #3 | Ushuaia
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Ushuaia
Processing Record 13; Set #3 | Sao Joao Da Barra
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Sao%20Joao%20Da%20Barra
Processing Record 14; Set #3 | Dikson
http://api.openw

Processing Record 14; Set #4 | Hilo
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Hilo
Processing Record 15; Set #4 | Gat
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Gat
Processing Record 16; Set #4 | Carnarvon
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Carnarvon
Processing Record 17; Set #4 | Taolanaro
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Taolanaro
Processing Record 18; Set #4 | Bethel
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Bethel
Processing Record 19; Set #4 | Port Alfred
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Port%20Alfred
Processing Record 20; Set #4 | Mar Del Plata
http://api.openweathermap.org/data/2.5/weather?appid=

Processing Record 19; Set #5 | Bredasdorp
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Bredasdorp
Processing Record 20; Set #5 | Nikolskoye
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Nikolskoye
Processing Record 21; Set #5 | Kapuskasing
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Kapuskasing
Processing Record 22; Set #5 | Nyrob
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Nyrob
Processing Record 23; Set #5 | Tuatapere
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Tuatapere
Processing Record 24; Set #5 | Saint-Philippe
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Saint-Philippe
Processing Record 25; Set #5 | Talnakh
http://api.openweathermap.o

...

Processing Record 43; Set #25 | Saint George
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Saint%20George
Processing Record 44; Set #25 | Hermanus
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Hermanus
Processing Record 45; Set #25 | Soyo
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Soyo
Processing Record 46; Set #25 | Mataura
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Mataura
Processing Record 47; Set #25 | Hasaki
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Hasaki
Processing Record 48; Set #25 | Rikitea
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Rikitea
Processing Record 49; Set #25 | Harper
http://api.openweathermap.org/data/2.5/weather?ap

Processing Record 46; Set #26 | Vaini
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Vaini
Processing Record 47; Set #26 | Constitucion
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Constitucion
Processing Record 48; Set #26 | Constitucion
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Constitucion
Processing Record 49; Set #26 | Gweta
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Gweta
Processing Record 50; Set #26 | Punta Arenas
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Punta%20Arenas
Processing Record 0; Set #27 | Znamenskoye
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Znamenskoye
Processing Record 1; Set #27 | Beringovskiy
http://api.openweat

Processing Record 48; Set #27 | Saryshagan
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Saryshagan
Processing Record 49; Set #27 | Yendi
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Yendi
Processing Record 50; Set #27 | Hermanus
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Hermanus
Processing Record 0; Set #28 | Tuktoyaktuk
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Tuktoyaktuk
Processing Record 1; Set #28 | Necochea
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Necochea
Processing Record 2; Set #28 | Ponta Do Sol
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Ponta%20Do%20Sol
Processing Record 3; Set #28 | Vardo
http://api.openweathermap.org/da

Processing Record 0; Set #29 | Bandarbeyla
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Bandarbeyla
Processing Record 1; Set #29 | Koygorodok
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Koygorodok
Processing Record 2; Set #29 | Busselton
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Busselton
Processing Record 3; Set #29 | Qaanaaq
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Qaanaaq
Processing Record 4; Set #29 | Barrow
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Barrow
Processing Record 5; Set #29 | Hermanus
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Hermanus
Processing Record 6; Set #29 | Hofn
http://api.openweathermap.org/data/2.5/weather

Processing Record 4; Set #30 | Atuona
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Atuona
Processing Record 5; Set #30 | Tsihombe
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Tsihombe
Processing Record 6; Set #30 | Palora
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Palora
Processing Record 7; Set #30 | Qaanaaq
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Qaanaaq
Processing Record 8; Set #30 | Hermanus
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Hermanus
Processing Record 9; Set #30 | Nemuro
http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q=Nemuro
Processing Record 10; Set #30 | Batagay-Alyta
http://api.openweathermap.org/data/2.5/weather?appid=cf6

In [18]:
print(len(cities))
print(len(lat))
print(len(lon))
print(len(temp_f))
print(len(humidity))
print(len(cloudiness))
print(len(wind_mph))

1500
1370
1370
1370
1370
1370
1370


In [34]:
weather_dict = {
    "City": cities,
    "Temp_Farenheit": temp_f,
    "Humidity": humidity,
    "Cloudiness": cloudiness,
    "Windspeed": wind_mph,
    "Latitude": lat,
    "Longitude": lon,
}


weather_data = pd.DataFrame.from_dict(weather_dict, orient='index')
weather_format = weather_data.transpose()
weather_format = weather_format.dropna(axis = 0, how ='any')
weather_format['Temp_Farenheit'] = weather_format['Temp_Farenheit'].map("{:,.2f}".format)
weather_format['Humidity'] = weather_format['Humidity'].map("{:,.2f}%".format)
weather_format['Cloudiness'] = weather_format['Cloudiness'].map("{:,.2f}%".format)
weather_format['Windspeed'] = weather_format['Windspeed'].map("{:,.2f}".format)
weather_format.index += 1 

print(f'The final data set is #{len(weather_format)} observations. Here are the first 5. See the csv file for the full set')
weather_format.head()


The final data set is #1370 observations. Here are the first 5. See the csv file for the full set


,City,Temp_Farenheit,Humidity,Cloudiness,Windspeed,Latitude,Longitude
1,Leningradskiy,5.06,78.00%,8.00%,6.60,69.38,178.42
2,Khatanga,29.72,91.00%,80.00%,12.19,71.98,102.47
3,Pangnirtung,8.60,65.00%,75.00%,2.24,66.15,-65.72
4,Punta Arenas,55.18,66.00%,75.00%,13.87,-53.16,-70.91
5,Mar Del Plata,75.35,28.00%,0.00%,11.63,-46.43,-67.52


In [ ]:
settings = {"units": "imperial", "appid": api_key}
current_weather = []

for city in cities:
    current_weather = owm.get_current(city, **settings)

print(f"Current weather object for {city}: {current_weather}.")

In [ ]:

current_weather_paris = owm.get_current({cities}, **settings)
print(f"Current weather object for Paris: {current_weather_paris}.")

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
# Save config information.
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

settings = {"units": "imperial", "appid": api_key}
current_weather_paris = owm.get_current("Paris", **settings)
print(f"Current weather object for Paris: {current_weather_paris}.")



# Build partial query URL
query_url = f"{url}appid={api_key}&units={units}&q="

for city in cities:
    response = requests.get(query_url + city).json()
    #  lat_lngs.append(response['coord']['lat'])
    #  temp.append(response['main']['temp'])

print(response)

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

#### Latitude vs. Humidity Plot

#### Latitude vs. Cloudiness Plot

#### Latitude vs. Wind Speed Plot